# Implementierung des GraphRAG-Systems

In [3]:
import os
import pandas as pd

from langchain_core.runnables import  RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from neo4j import GraphDatabase
from langchain_neo4j import Neo4jGraph
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI



load_dotenv()

True

### Verbindung mit dem Knowledge Graphen herstellen

#### Eine Intsanz wurde mit Neo4j Aura erstellt und mit dem Knowledge Grpah Builder von Neo4j verbunden

In [2]:
uri = os.getenv("NEO4J_URI")  
username = os.getenv("NEO4J_USERNAME")
password = os.getenv("NEO4J_PASSWORD")

In [3]:
graph = Neo4jGraph(url=uri, username=username, password=password)

# LLM instanzieren

In [4]:
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    openai_api_key=os.environ["OPEN_AI_API_KEY"]
)

### Fulltext index wird erstellt

In [12]:
# Der folgende Code stammt aus dem Repository: GraphRAG-with-Llama-3.1

# Quelle: https://github.com/Coding-Crashkurse/GraphRAG-with-Llama-3.1/blob/main/enhancing_rag_with_graph.ipynb


driver = GraphDatabase.driver(
        uri = uri,
        auth = (username, password))

def create_fulltext_index(tx):
    query = '''
    CREATE FULLTEXT INDEX `fulltext_entity_id` 
    FOR (n:__Entity__) 
    ON EACH [n.id];
    '''
    tx.run(query)

# Function to execute the query
def create_index():
    with driver.session() as session:
        session.execute_write(create_fulltext_index)
        print("Fulltext index created successfully.")

# Call the function to create the index
try:
    create_index()
except:
    pass

# Close the driver connection
driver.close()

Fulltext index created successfully.


### Extrahiert die Entities aus der Frage

In [6]:
with open("../prompts/entity_chain_prompt.txt", "r", encoding="utf-8") as e:
    entity_prompt = e.read()

In [8]:
prompt_extrahiere_entitaet = ChatPromptTemplate.from_template(
    f"{entity_prompt}"
)

entity_chain = (
    {"frage": RunnablePassthrough()}
    | prompt_extrahiere_entitaet
    | llm
    | StrOutputParser()
    | (lambda x: [name.strip() for name in x.split(",")])
)


In [9]:
entity_chain.invoke("Wie bezeichnete Donald Trump in seiner Wahlkampagne den Klimawandel?")

['Donald Trump', 'Klimawandel']

In [ ]:
# Der folgende Code ist eine Anlehnung an den Code aus dem Repository: GraphRAG-with-Llama-3.1

# Quelle: https://github.com/Coding-Crashkurse/GraphRAG-with-Llama-3.1/blob/main/enhancing_rag_with_graph.ipynb

def graph_retriever(question: str):
    """
    Gibt die Triplets zurück

    """
    result = ""
    # Entitäten aus der Frage Extrahieren
    entities = entity_chain.invoke(question)
    # Über die Entitäten iterieren
    for entity in entities:
        # Cypher Abfrage an den Knowledge Graph, dass die ausgehnden und eingehenden Relationen gefunden werden und als Triplet zurückgegben werden 
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('fulltext_entity_id', $query, {limit:2})
                YIELD node, score
                CALL {
                WITH node
                MATCH (node)-[r]->(neighbor)
                WHERE type(r) <> 'HAS_ENTITY'
                RETURN DISTINCT node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
                UNION ALL
                WITH node
                MATCH (node)<-[r]-(neighbor)
                WHERE type(r) <> 'HAS_ENTITY'
                RETURN DISTINCT neighbor.id + ' - ' + type(r) + ' -> ' + node.id AS output
                }
                RETURN output LIMIT 50
    
            """,
            {"query": entity},
        )
        result += "\n".join([el['output'] for el in response])
    return result

## Prompt für das LLM

In [12]:
with open("../prompts/graph_rag.txt", "r", encoding="utf-8") as g:
    graph_rag_prompt = g.read()

In [ ]:
template = f"{graph_rag_prompt}"

prompt = ChatPromptTemplate.from_template(template)

### Chain für das GraphRAG-System

In [89]:
chain_graphRAG = (
    {
        "context": graph_retriever,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [92]:
chain_graphRAG.invoke("Wie bezeichnete Donald Trump in seiner Wahlkampagne den Klimawandel?")

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 3, column: 1, offset: 93} for query: "CALL db.index.fulltext.queryNodes('fulltext_entity_id', $query, {limit:2})\nYIELD node, score\nCALL {\n  WITH node\n  MATCH (node)-[r]->(neighbor)\n  WHERE type(r) <> 'HAS_ENTITY'\n  RETURN DISTINCT node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n  UNION ALL\n  WITH node\n  MATCH (node)<-[r]-(neighbor)\n  WHERE type(r) <> 'HAS_ENTITY'\n  RETURN DISTINCT neighbor.id + ' - ' + type(r) + ' -> ' + node.id AS output\n}\nRETURN output LIMIT 50\n\n            "
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category:

'Donald Trump bezeichnete in seiner Wahlkampagne den Klimawandel als "großen Schwindel".'

In [ ]:


def run_graphrag_antworten(csv_path, save_path):
    """
    Die Fragen werden dem System gestellt, die Antworten werden 
    generiert und als neue Spalte zu der CSV Datei hinzugefügt

    """
    # CSV mit den Fragen Laden diese enthaelt schon die Antworten des VectorRAG-systems
    df = pd.read_csv(csv_path)

    # Spalte für Antworten vorbereiten
    antworten = []
    #Über die Zeilen des Dfs iterieren
    for _, row in df.iterrows():
        # Die Frage aus der Zeile extrahieren
        frage = row["frage"]
        try:
            # GraphRAG-Chain wird aufgerufen
            antwort = chain_graphRAG.invoke(frage)
        except Exception as e:
            print(f"Fehler bei Artikel {row['artikel_id']} Frage {row['frage_nr']}: {e}")
            antwort = "FEHLER"
        # Antwort zu der liste Hinzufügen    
        antworten.append(antwort)
    # Spalte mit den Antworten des Systems zum Dataframe hinzufügen
    df["graph_rag_antwort_original_3"] = antworten

    # Df als CSV speichern
    if save_path:
        df.to_csv(save_path, index=False, encoding="utf-8")
        print(f"Ergebnisse gespeichert unter: {save_path}")

    return df

In [ ]:


df_graphRAG_antworten = run_graphrag_antworten("../antworten_der_systeme_manipuliert_beide_durchgänge/vectorrag_antworten_original_2.csv",
                                               "../antworten_der_systeme_originalkontext_beide_durchgänge/graph_rag_und_vector_rag_antworten_original_2.csv" 
                                               )

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 3, column: 1, offset: 93} for query: "CALL db.index.fulltext.queryNodes('fulltext_entity_id', $query, {limit:2})\nYIELD node, score\nCALL {\n  WITH node\n  MATCH (node)-[r:!HAS_ENTITY]->(neighbor)\n  RETURN DISTINCT node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n  UNION ALL\n  WITH node\n  MATCH (node)<-[r:!HAS_ENTITY]-(neighbor)\n  RETURN DISTINCT neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output\n}\nRETURN output LIMIT 50;\n\n            "
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is de

Fehler bei Artikel 47 Frage 1: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `db.index.fulltext.queryNodes`: Caused by: org.apache.lucene.queryparser.classic.TokenMgrError: Lexical error at line 1, column 24.  Encountered: <EOF> after prefix "/Edison Research" (in lexical state 2)}


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 3, column: 1, offset: 93} for query: "CALL db.index.fulltext.queryNodes('fulltext_entity_id', $query, {limit:2})\nYIELD node, score\nCALL {\n  WITH node\n  MATCH (node)-[r:!HAS_ENTITY]->(neighbor)\n  RETURN DISTINCT node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n  UNION ALL\n  WITH node\n  MATCH (node)<-[r:!HAS_ENTITY]-(neighbor)\n  RETURN DISTINCT neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output\n}\nRETURN output LIMIT 50;\n\n            "
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is de

Ergebnisse gespeichert unter: C:\bachelor\experiment_bachelor_lb\graph_rag_und_vector_rag_antworten_original_2_3.csv


,artikel_id,frage_nr,frage,musterloesung,fake_news_art,vector_rag_antwort_original_2,graph_rag_antwort_original_3
0,10,1,Wer wurde von Trump für das Amt des Justizmini...,Der erzkonservative Abgeordnete Matt Gaetz wur...,Fake Fact,Donald Trump hat den erzkonservativen Abgeordn...,Der bereitgestellte Kontext enthält keine spez...
1,10,2,Wie wird Gaetz in Bezug auf seine Verbindung z...,Gaetz gilt als Trump-Loyalist und einer der ve...,Framing,Gaetz wird in Bezug auf seine Verbindung zu Tr...,Gaetz wird in Bezug auf seine Verbindung zu Tr...
2,10,3,Wie wurde die Nominierung von Gaetz zum Justiz...,Die Nominierung von Gaetz zum Justizminister w...,Exaggeration,Der bereitgestellte Kontext enthält keine Info...,Der bereitgestellte Kontext enthält keine Info...
3,11,1,"Wer wurde von Donald Trump als ""Grenzzar"" erna...","Tom Homan wurde von Donald Trump als ""Grenzzar...",Fake Fact,"Donald Trump hat Tom Homan als ""Grenzzar"" erna...",Der bereitgestellte Kontext enthält keine Info...
4,11,2,"Wie hat Trump seine Entscheidung für den ""Gren...",Trump begründete seine Entscheidung für Tom Ho...,Framing,"Donald Trump hat seine Entscheidung, Tom Homan...",Der bereitgestellte Kontext enthält keine spez...
...,...,...,...,...,...,...,...
280,95,2,Wie hat Meta seinen Schritt gegen die russisch...,"Meta hat den Schritt mit dem Vorwurf der ""ausl...",Framing,Meta hat seinen Schritt gegen die russischen S...,Meta hat seinen Schritt gegen die russischen S...
281,95,3,Wie ist der aktuelle Status von RT auf der Pla...,Auf der Plattform X (ehemals Twitter) sind die...,Exaggeration,"Auf der Plattform X, ehemals Twitter, sind die...",RT ist auf der Plattform X außerhalb der EU ve...
282,9,1,Welche Rolle hat Mike Waltz in Bezug auf den K...,Mike Waltz ist als Verfechter eines harten Kur...,Fake Fact,Mike Waltz hat eine bedeutende Rolle in Bezug ...,Mike Waltz spielt eine bedeutende Rolle in Bez...
283,9,2,Wie wurde Mike Waltz' Meinung zu China im Orig...,Im Originalartikel wurde Mike Waltz' Haltung a...,Framing,Mike Waltz' Meinung zu China wurde im Original...,Im Originalartikel wurde Mike Waltz' Meinung z...
